In [3]:
# Colab Cell 2 — Installer paquets
!pip install -q streamlit==1.22.0 pyngrok tensorflow Pillow numpy
# (la version de tensorflow peut être ajustée si nécessaire)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires p

In [7]:
# Colab Cell 5 — Lancer Streamlit en arrière-plan et créer tunnel ngrok
import os, time, subprocess, sys
from pyngrok import ngrok

# --- CONFIG ---
NGROK_AUTHTOKEN = "2vdJFZExAVhUhUeOJZjhTHOLFhG_3noX2p3A3rDQ3KYjAKFgF"  # remplace si nécessaire
PORT = 8501
LOGFILE = "/content/streamlit.log"

# 1) Auth token ngrok
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 2) Kill d'anciennes instances (sécuritaire)
try:
    !pkill -f streamlit
except Exception:
    pass

time.sleep(1)

# 3) Lancer streamlit dans un subprocess (headless)
cmd = [sys.executable, "-m", "streamlit", "run", "app.py", "--server.port={}".format(PORT), "--server.headless=true"]
print("Lancement de Streamlit :", " ".join(cmd))
f = open(LOGFILE, "wb")
proc = subprocess.Popen(cmd, stdout=f, stderr=subprocess.STDOUT)

# 4) Attendre un peu que streamlit démarre
print("PID Streamlit:", proc.pid)
print("Attente de démarrage (10s)...")
time.sleep(10)

# 5) Créer tunnel ngrok
public_url = ngrok.connect(PORT).public_url
print("Streamlit public URL:", public_url)
print("Logs: ", LOGFILE)


Lancement de Streamlit : /usr/bin/python3 -m streamlit run app.py --server.port=8501 --server.headless=true
PID Streamlit: 949
Attente de démarrage (10s)...
Streamlit public URL: https://c2c0c894df4c.ngrok-free.app
Logs:  /content/streamlit.log


In [6]:
!pkill -f ngrok && fuser -k 8501/tcp && echo "✅ Nettoyage terminé"


^C


In [4]:
%%writefile app.py
import streamlit as st
from PIL import Image, ImageOps
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt
import pandas as pd
import io
import traceback
#VERSIONN FINAAALEEE
# ----- CONFIG PATHS -----
DENSENET_PATH = "/content/drive/MyDrive/TumourDL/densenet_brain_tumor.h5"
VGG16_PATH = "/content/drive/MyDrive/TumourDL/vgg16_brain_tumor.h5"
RESNET_PATH = "/content/drive/MyDrive/Copie de resnet152v2_brain_tumor.h5"
# Ton fichier Xception — peut être .keras (full model) ou un fichier de weights compatible
XCEPTION_PATH = "/content/drive/MyDrive/TumourDL/xception_brain_tumor.keras"

# ----- SETTINGS -----
IMG_SIZE = (224, 224)
XCEPTION_SIZE = (299, 299)
CLASS_NAMES = ['glioma', 'meningioma', 'notumor', 'pituitary']

st.set_page_config(page_title="🧠 Brain Tumor Classifier", layout="wide", page_icon="🧠")
st.title("🧠 Brain Tumor Classifier — DenseNet121, VGG16, ResNet152V2 & Xception")
st.markdown("Upload an image, choose un modèle, et voir les prédictions et comparatifs.")

# ----- Robust Xception loader -----
def try_load_xception(path, xception_size=XCEPTION_SIZE, num_classes=len(CLASS_NAMES)):
    """
    1) Try load_model(path) — works if path is a full saved model (.keras or full .h5 model)
    2) If that fails, rebuild Xception with input_shape=(299,299,3) and try model.load_weights(path)
    """
    tb_full = tb_weights = ""
    # 1) try load_model (full model)
    try:
        m = load_model(path)
        return m
    except Exception as e_full:
        tb_full = traceback.format_exc()
        # continue to try weights approach

    # 2) try rebuild architecture and load weights
    try:
        base = Xception(weights=None, include_top=False, input_shape=(xception_size[0], xception_size[1], 3))
        x = layers.GlobalAveragePooling2D()(base.output)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        outputs = layers.Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=outputs)
        model.load_weights(path)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    except Exception as e_weights:
        tb_weights = traceback.format_exc()

    # both failed => raise informative RuntimeError
    raise RuntimeError(
        "Both load_model() and rebuild+load_weights() failed for Xception.\n\n"
        f"load_model() traceback:\n{tb_full}\n\nrebuild+load_weights() traceback:\n{tb_weights}\n\n"
        "Causes possibles:\n"
        "- Le fichier n'est ni un modèle complet ni un fichier de poids compatibles.\n"
        "- Le fichier de poids a été sauvegardé depuis une architecture différente.\n\n"
        "Solutions recommandées:\n"
        "- Si tu as le script d'entraînement : `model.save('/path/xception_brain_tumor.keras')` (modèle complet).\n"
        "- Ou sauvegarde les poids avec `model.save_weights('/path/xception_weights.h5')` et fournis ces poids.\n"
    )

# ----- Cached loader -----
@st.cache_resource(show_spinner=True)
def load_model_cached(path, model_name=None):
    if model_name == "Xception":
        return try_load_xception(path)
    return load_model(path)

# ----- Load models -----
dense_model = vgg_model = resnet_model = xception_model = None
missing = []

try:
    dense_model = load_model_cached(DENSENET_PATH)
except Exception as e:
    missing.append(("DenseNet121", DENSENET_PATH, str(e)))

try:
    vgg_model = load_model_cached(VGG16_PATH)
except Exception as e:
    missing.append(("VGG16", VGG16_PATH, str(e)))

try:
    resnet_model = load_model_cached(RESNET_PATH)
except Exception as e:
    missing.append(("ResNet152V2", RESNET_PATH, str(e)))

try:
    xception_model = load_model_cached(XCEPTION_PATH, model_name="Xception")
except Exception as e:
    missing.append(("Xception", XCEPTION_PATH, str(e)))

if missing:
    st.warning("Certains modèles n'ont pas pu être chargés — vérifie les chemins et formats. Détails :")
    for name, path, err in missing:
        st.markdown(f"**{name}** — `{path}`")
        # show first lines of error to keep UI compact
        first_lines = "\n".join(str(err).splitlines()[:12])
        st.code(first_lines, language="text")
        st.markdown("---")
    st.info("Si possible, fournis un modèle complet `.keras` ou un fichier de poids compatible et la même architecture.")

# ----- Preprocessing (adaptée au modèle choisi) -----
def preprocess_image_for_model(img: Image.Image, model_name=None):
    """
    Convertit en RGB, ajuste la taille en fonction du modèle (Xception=299, autres=224), normalise et ajoute batch dim.
    """
    if img.mode != "RGB":
        img = img.convert("RGB")

    if model_name == "Xception":
        target_size = XCEPTION_SIZE
    else:
        target_size = IMG_SIZE

    img = ImageOps.fit(img, target_size, method=Image.Resampling.LANCZOS)
    arr = np.array(img).astype("float32") / 255.0
    arr = np.expand_dims(arr, axis=0)
    return arr

# ----- Prediction (utilise model_choice) -----
def predict_model(model, img, model_name=None):
    if model is None:
        raise ValueError("Model is None — impossible de prédire.")
    x = preprocess_image_for_model(img, model_name=model_name)
    preds = model.predict(x)
    if preds.ndim == 2:
        preds = preds[0]
    top_idx = preds.argsort()[::-1]
    return preds, top_idx

# ----- Sidebar -----
st.sidebar.header("⚙️ Options")
show_topk = st.sidebar.slider("Afficher top-k probabilités", 1, len(CLASS_NAMES), 3)
threshold = st.sidebar.slider("Seuil d'affichage (probabilité minimale)", 0.0, 1.0, 0.0, 0.01)

# ----- Upload + Prediction -----
st.header("1️⃣ Upload & Prédiction")
uploaded_file = st.file_uploader("Choisis une image (jpg/png)", type=["jpg", "jpeg", "png"])

if uploaded_file:
    try:
        image = Image.open(io.BytesIO(uploaded_file.read()))
    except Exception as e:
        st.error(f"Erreur lors de la lecture de l'image : {e}")
        st.stop()

    col1, col2 = st.columns([1,1])
    with col1:
        st.image(image, caption="Image uploadée", use_column_width=True)

    with col2:
        available_models = []
        if dense_model is not None: available_models.append("DenseNet121")
        if vgg_model is not None: available_models.append("VGG16")
        if resnet_model is not None: available_models.append("ResNet152V2")
        if xception_model is not None: available_models.append("Xception")

        if not available_models:
            st.error("Aucun modèle disponible — corrige les chemins ou formats et relance.")
        else:
            model_choice = st.radio("Choisir le modèle pour la prédiction", available_models)
            model = {
                "DenseNet121": dense_model,
                "VGG16": vgg_model,
                "ResNet152V2": resnet_model,
                "Xception": xception_model
            }[model_choice]

            preds, top_idx = predict_model(model, image, model_name=model_choice)

            st.markdown("### Top probabilités :")
            for i in range(show_topk):
                idx = top_idx[i]
                prob = float(preds[idx])
                label = CLASS_NAMES[idx]
                if prob >= threshold:
                    st.success(f"{i+1}. {label} — {prob*100:.2f}%")
                else:
                    st.warning(f"{i+1}. {label} — {prob*100:.2f}% (sous seuil)")

            best_i = top_idx[0]
            best_prob = float(preds[best_i])
            best_label = CLASS_NAMES[best_i]
            if best_prob >= threshold:
                st.success(f"✅ Prédiction principale: {best_label} ({best_prob*100:.2f}%)")
            else:
                st.warning(f"⚠️ Prédiction principale: {best_label} ({best_prob*100:.2f}%) sous le seuil")

# ----- Comparatif -----
st.header("2️⃣ Comparatif des modèles")
comparative_data = {
    "Modèle": ["DenseNet121", "VGG16", "ResNet152V2", "Xception"],
    "Accuracy Test (%)": [83.83, 88.25, 93.97, 70.00],
    "Précision (%)": [84.0, 88.5, 94.40, 68.70],
    "Recall (%)": [83.0, 87.5, 93.97, 70.00]
}
df_compare = pd.DataFrame(comparative_data)
st.table(df_compare)


st.markdown("---")
st.caption("Made with Streamlit 🧠")


Writing app.py
